In [75]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import wfdb
from utils import qrs_detect, comp_cosEn, save_dict
from entry_2021 import load_data
from buffer import buffer 
from utils_dawid import extract_basic_features, butter_bandpass_filter
import xgboost as xgb

In [60]:
DATA_PATH = 'C:\\Users\\ASHISH\\Python_data_code\\Ashish\\python_entry_by_nidhi\\New_folder'

In [61]:
test_set = open(os.path.join(DATA_PATH, 'RECORDS'), 'r').read().splitlines()

In [62]:
for i, sample in enumerate(test_set):
        print(sample)
        sample_path = os.path.join(DATA_PATH, sample)

Training_set_I/data_30_2
Training_set_I/data_35_8
Training_set_I/data_31_1
Training_set_I/data_9_5
Training_set_I/data_53_2
Training_set_I/data_31_3
Training_set_I/data_18_3
Training_set_I/data_41_10
Training_set_I/data_31_13
Training_set_I/data_17_14
Training_set_I/data_44_17
Training_set_I/data_43_8
Training_set_I/data_8_10
Training_set_I/data_10_5
Training_set_I/data_34_16
Training_set_I/data_9_9
Training_set_I/data_47_12
Training_set_I/data_47_13
Training_set_I/data_26_6
Training_set_I/data_38_4
Training_set_I/data_33_18
Training_set_I/data_32_4
Training_set_I/data_46_7
Training_set_I/data_41_14
Training_set_I/data_41_9
Training_set_I/data_41_8
Training_set_I/data_48_12
Training_set_I/data_23_2
Training_set_I/data_26_11
Training_set_I/data_43_4
Training_set_I/data_32_17
Training_set_I/data_53_11
Training_set_I/data_33_14
Training_set_I/data_24_26
Training_set_I/data_8_24
Training_set_I/data_15_10
Training_set_I/data_0_6
Training_set_I/data_46_12
Training_set_I/data_26_5
Training_se

In [64]:
def load_data(sample_path):
    sig, fields = wfdb.rdsamp(sample_path)
    length = len(sig)
    fs = fields['fs']

    return sig, length, fs

def Label_extraction(sample_path, sample_descrip, signal_length):
    ann_ref = wfdb.rdann(sample_path, 'atr')
    beat_loc = np.array(ann_ref.sample) # r-peak locations
    ann_note = np.array(ann_ref.aux_note) # rhythm change flag
    sig_lab=np.zeros(signal_length)

    if 'non atrial fibrillation' in sample_descrip:
                class_true = 0
    elif 'persistent atrial fibrillation' in sample_descrip:
                class_true = 1
                sig_lab=np.ones(signal_length)
    elif 'paroxysmal atrial fibrillation' in sample_descrip:
                class_true = 2
                af_start_scripts = np.where((ann_note=='(AFIB') | (ann_note=='(AFL'))[0]
                af_end_scripts = np.where(ann_note=='(N')[0]
                for j in range(len(af_start_scripts)):
                    aa=af_start_scripts[j]
                    bb=af_end_scripts[j]
                    sig_lab[beat_loc[aa]:beat_loc[bb],]=1
    else:
        print('Error: the recording is out of range!')
    return sig_lab, class_true

def ECG_segmentation_test(sig, sig_lab, fs, windowLen,shiftLen):
    
    
    duration=int(windowLen*fs)
    dataOverlap = int((windowLen-shiftLen)*fs)           
    bufOut_sig=buffer(sig,duration,dataOverlap) 
    bufOut_lab=buffer(sig_lab,duration,dataOverlap)
    
    bufOut_lab_1=  np.sum(bufOut_lab,axis=1)> dataOverlap
    #bufOut_lab_1=  np.sum(bufOut_lab,axis=1)> 5
    bufOut_lab_1=np.array(bufOut_lab_1)+0 # convert bool to int
    return bufOut_sig, bufOut_lab_1

def Feature_extraction_test(bufOut_sig, bufOut_lab_1):
    feat_11=[]
    lab_all=[]
    for j in range(len(bufOut_sig)):
        seg_temp=bufOut_sig[j,:]
        lab_temp=bufOut_lab_1[j]      
        feat_1=extract_basic_features(seg_temp, 30000)
        # feat_2
        # feat_3        
        
        feat_11.append(feat_1)
        lab_all.append(lab_temp)   
    
    return feat_11, lab_all

In [65]:
labels=[]
windowLen=30 # segmentation    
shiftLen=25 # segmentation  
UpTh=0.75
LwTh=0.20
complete_features=[]
complete_labels=[]
for i, sample in enumerate(test_set[0:10]):
        print(sample)
        sample_path = os.path.join(DATA_PATH, sample)
        sig, L, fs = load_data(sample_path)
        sig, fields = wfdb.rdsamp(sample_path)
        ann_ref = wfdb.rdann(sample_path, 'atr')
        sample_descrip = fields['comments']
        if 'non atrial fibrillation' in sample_descrip:
            class_true = 0
        elif 'persistent atrial fibrillation' in sample_descrip:
            class_true = 1
        elif 'paroxysmal atrial fibrillation' in sample_descrip:
            class_true = 2
        else:
            print('Error: the recording is out of range!')
        labels.append(class_true)
        sig = sig[:, 1]
        signal_length=len(sig)
        sig_lab, class_true=Label_extraction(sample_path, sample_descrip, signal_length)
        bufOut_sig_test, bufOut_lab_1 = ECG_segmentation_test(sig, sig_lab, fs, windowLen,shiftLen)
        feat_11_test, lab = Feature_extraction_test(bufOut_sig_test,  bufOut_lab_1)
        complete_features.append(np.vstack(feat_11_test))
        complete_labels.append(np.vstack(lab))
        
        
        
        
        

Training_set_I/data_30_2


C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountere

C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountere

C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountere

C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered

...

Training_set_I/data_31_13
Training_set_I/data_17_14


C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered

C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered

C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered

C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered in multiply
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: divide by zero encountered in log
  shannon_energy = -square_decg_power*np.log(square_decg_power)
C:\Users\ASHISH\Python_data_code\Ashish\cpsc2021-python-entry-main\rpeakdetectr\rpeakdetect.py:76: RuntimeWarning: invalid value encountered

In [74]:
feat_array=np.vstack(complete_features)
lab_array=np.vstack(complete_labels)

In [72]:

lab_array

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [8]:
model = xgb.XGBClassifier()           
model.fit(feat_array, lab_array)

In [ ]:
y_pred = model.predict(X_data)